In [1]:
!nvidia-smi

Fri Dec 15 08:30:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                On  | 00000000:86:00.0 Off |                    0 |
| N/A   36C    P0             ERR! / 250W |      0MiB / 46080MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os

In [3]:
os.getcwd()

'/scratch/aa9798'

In [4]:
!/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade --target=/scratch/aa9798 nltk
!/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade --target=/scratch/aa9798 sacrebleu
!/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade --target=/scratch/aa9798 Rouge


  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2023.10.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached regex-2023.10.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (776 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 20.2.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached sacrebleu-2.4.0-py3-none-any.whl.metadata (57 kB)
  Using cached portalocker-2.8.2-py3-none-any.whl.metadata (8.5 kB)
  Using cached regex-2023.10.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Us

In [5]:
!/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade --target=/scratch/aa9798 sentencepiece


  Using cached sentencepiece-0.1.99-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)

[notice] A new release of pip is available: 20.2.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
!/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade --target=/scratch/aa9798 transformers wandb -q


[notice] A new release of pip is available: 20.2.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade --target=/scratch/aa9798 torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.1.2%2Bcu118-cp38-cp38-linux_x86_64.whl (2325.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.16.2%2Bcu118-cp38-cp38-linux_x86_64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.1.2%2Bcu118-cp38-cp38-linux_x86_64.whl (3.2 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.0-py3-none-any.whl (2.0 MB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Using cached https://download.pytorch.org/whl/triton-2.1.0-

In [ ]:
!/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade --target=/scratch/aa9798 scikit-learn


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import io


from transformers import T5Tokenizer, T5ForConditionalGeneration
import wandb

In [ ]:
import math
import collections
import numpy as np
import pandas as pd
from rouge import Rouge
from rouge import Rouge
rouge = Rouge()
def compute_bleu(reference_corpus,
                 translation_corpus,
                 max_order=1,
                 smooth=False):
    def _get_ngrams(segment, max_order):
        ngram_counts = collections.Counter()
        for order in range(1, max_order + 1):
            for i in range(0, len(segment) - order + 1):
                ngram = tuple(segment[i:i + order])
                ngram_counts[ngram] += 1
        return ngram_counts

    matches_by_order = [0] * max_order
    possible_matches_by_order = [0] * max_order
    reference_length = 0
    translation_length = 0
    for (references, translation) in zip(reference_corpus, translation_corpus):
        reference_length += min(len(r) for r in references)
        translation_length += len(translation)

        merged_ref_ngram_counts = collections.Counter()
        for reference in references:
            merged_ref_ngram_counts |= _get_ngrams(reference, max_order)
        translation_ngram_counts = _get_ngrams(translation, max_order)
        overlap = translation_ngram_counts & merged_ref_ngram_counts
        for ngram in overlap:
            matches_by_order[len(ngram) - 1] += overlap[ngram]
        for order in range(1, max_order + 1):
            possible_matches = len(translation) - order + 1
            if possible_matches > 0:
                possible_matches_by_order[order - 1] += possible_matches

    precisions = [0] * max_order
    for i in range(0, max_order):
        if smooth:
            precisions[i] = ((matches_by_order[i] + 1.) /
                             (possible_matches_by_order[i] + 1.))
        else:
            if possible_matches_by_order[i] > 0:
                precisions[i] = (float(matches_by_order[i]) /
                                 possible_matches_by_order[i])
            else:
                precisions[i] = 0.0

    if min(precisions) > 0:
        p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
        geo_mean = math.exp(p_log_sum)
    else:
        geo_mean = 0

    ratio = float(translation_length) / reference_length

    if ratio > 1.0:
        bp = 1.
    else:
        bp = math.exp(1 - 1. / ratio)

    bleu = geo_mean * bp

    return bleu

def compute_rouge(reference_corpus, translation_corpus):
    rouge = Rouge()
    scores = rouge.get_scores(translation_corpus, reference_corpus, avg=True)
    return scores

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    total_loss = 0
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        total_loss += loss.item()
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return total_loss/len(loader)

In [ ]:
def train_and_validate(epoch, tokenizer, model, device, train_loader, val_loader, optimizer):
    model.train()
    total_train_loss = 0

    for _, data in enumerate(train_loader, 0):
        # Training code (as in your existing train function)
        y = data['target_ids'].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype=torch.long)
        mask = data['source_mask'].to(device, dtype=torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        total_train_loss += loss.item()

        if _ % 10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _ % 10 == 0:
            print(f'Epoch: {epoch}, Training Loss: {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate average training loss for the epoch
    avg_train_loss = total_train_loss / len(train_loader)

    # Print and log training loss
    print(f'Epoch: {epoch}, completed Training Loss: {avg_train_loss}')
    wandb.log({"Train Loss": avg_train_loss})

    # Validation step
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for _, val_data in enumerate(val_loader, 0):
            # Validation code (similar to your existing validate function)
            # Make sure to accumulate the validation loss
            val_y = val_data['target_ids'].to(device, dtype=torch.long)
            val_y_ids = val_y[:, :-1].contiguous()
            val_labels = val_y[:, 1:].clone().detach()
            val_labels[val_y[:, 1:] == tokenizer.pad_token_id] = -100
            val_ids = val_data['source_ids'].to(device, dtype=torch.long)
            val_mask = val_data['source_mask'].to(device, dtype=torch.long)

            val_outputs = model(input_ids=val_ids, attention_mask=val_mask, decoder_input_ids=val_y_ids, labels=val_labels)
            val_loss = val_outputs[0]
            total_val_loss += val_loss.item()

    # Calculate average validation loss for the epoch
    avg_val_loss = total_val_loss / len(val_loader)

    # Print and log validation loss
    print(f'Epoch: {epoch}, completed Validation Loss: {avg_val_loss}')
    wandb.log({"Validation Loss": avg_val_loss})

    return avg_train_loss, avg_val_loss  # You can return other metrics if needed


In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    total_iterations = len(loader)
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype=torch.long)
            ids = data['source_ids'].to(device, dtype=torch.long)
            mask = data['source_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
            )

            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]

            if _ % 10 == 0:
                print(_)

            if _ % 100 == 0:
                print(f'Completed {_} iterations')

            predictions.extend(preds)
            actuals.extend(target)

    return predictions, actuals


In [ ]:
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Function to read files and create DataFrame
def create_dataframe(root_folder):
    data = {'articles': [], 'summaries': [], 'category': [], 'file_name': []}

    # Loop through the subfolders
    for category in ['business', 'entertainment', 'politics', 'sport', 'tech']:
        # Path to News Articles and Summaries folders
        articles_path = os.path.join(root_folder, 'News Articles', category)
        summaries_path = os.path.join(root_folder, 'Summaries', category)

        # Get the list of files in the category
        files_list = [file_name for file_name in os.listdir(articles_path) if file_name.endswith('.txt')]

        # Loop through the files in each subfolder with a progress bar
        for file_name in tqdm(files_list, desc=f'Processing {category}'):
            # Read article content
            with open(os.path.join(articles_path, file_name), 'r', encoding='utf-8', errors='replace') as article_file:
                article_content = article_file.read()

            # Read summary content
            summary_file_path = os.path.join(summaries_path, file_name)
            with open(summary_file_path, 'r', encoding='utf-8') as summary_file:
                summary_content = summary_file.read()

            # Append data to dictionary
            data['articles'].append(article_content)
            data['summaries'].append(summary_content)
            data['category'].append(category)
            data['file_name'].append(file_name)

    # Create DataFrame
    df = pd.DataFrame(data)
    return df




# Replace 'path_to_your_folder' with the actual path to your 'BBC News Summary' folder
df = create_dataframe('/scratch/aa9798/news')




In [ ]:
old_df = df.copy()

In [ ]:
df = df.rename(columns={'articles': 'text', 'summaries': 'ctext'})

In [ ]:
print("Maximum length of text in 'main_text' column:", df['text'].apply(len).max())
print("Maximum length of text in 'main_text' column:", df['ctext'].apply(len).max())

In [ ]:
wandb.init(project="transformers_tutorials_summarization")
config = wandb.config
config.TRAIN_BATCH_SIZE = 1    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 1    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 10       # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)
config.MAX_LEN = 4000
config.SUMMARY_LEN = 1500

In [ ]:
# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(config.SEED) # pytorch random seed
np.random.seed(config.SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

In [ ]:
train_size = 0.6
validation_size = 0.2
test_size = 0.2

# Split into train and temp sets
train_dataset, temp_dataset = train_test_split(df, test_size=1 - train_size, random_state=config.SEED)

# Split the temp set into validation and test sets
val_dataset, test_dataset = train_test_split(temp_dataset, test_size=test_size/(validation_size + test_size), random_state=config.SEED)

# Resetting indices for all datasets
train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

In [ ]:

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VAL Dataset: {}".format(val_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))


## SET T5 PARAMETERS

In [ ]:
training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
test_set = CustomDataset(test_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

test_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
test_loader = DataLoader(test_set, **val_params)


## TRAIN T5 MODEL

In [ ]:
# Define the T5 model and move it to the appropriate device
model = T5ForConditionalGeneration.from_pretrained("t5-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Set up optimizer and log the model with wandb
train_losses = []
val_losses = []
optimizer = torch.optim.Adam(params=model.parameters(), lr=config.LEARNING_RATE)
wandb.watch(model, log="all")

# Start fine-tuning loop
print('Initiating Fine-Tuning for the model on our dataset')
for epoch in range(config.TRAIN_EPOCHS):
    # Train and validate the model for each epoch
    train_loss, val_loss = train_and_validate(epoch, tokenizer, model, device, training_loader, val_loader, optimizer)
    
    # Append losses to lists and print the train loss for the epoch
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print('Epoch:', epoch + 1, '| Train Loss:', train_loss, '| Val Loss:', val_loss)
    print('Evaluating on the validation dataset...')
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    rouge_scores = compute_rouge(actuals, predictions)

    # Print Rouge scores
    print('Epoch:', epoch + 1, '| Rouge-1:', round(rouge_scores['rouge-1']['f'], 4))
    print('Epoch:', epoch + 1, '| Rouge-2:', round(rouge_scores['rouge-2']['f'], 4))
    print('Epoch:', epoch + 1, '| Rouge-L:', round(rouge_scores['rouge-l']['f'], 4))
    # Assuming you have a function to compute BLEU
    bleu_score = compute_bleu(actuals, predictions)
    print('Epoch:', epoch + 1, '| BLEU:', round(bleu_score, 4))
    

save_directory = "/scratch/aa9798/best_model"
model.save_pretrained(save_directory)

# Optionally, save the tokenizer as well if needed
model_tokenizer = T5Tokenizer.from_pretrained("t5-base")
model_tokenizer.save_pretrained(save_directory)

# Print a message indicating the start of summary generation
print('Now generating summaries on our fine-tuned model for the validation dataset and saving it in a dataframe')

# Start validation loop for generating summaries
for epoch in range(config.VAL_EPOCHS):
    # Validate the model and get predictions and actuals
    predictions, actuals = validate(epoch, tokenizer, model, device, test_loader)
    
    # Create a dataframe with generated and actual text, and print a message
    final_df = pd.DataFrame({'Generated Text': predictions, 'Actual Text': actuals})
    print('Output Files generated for review')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/scratch/aa9798/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Training Loss: 12.617023468017578
Epoch: 0, Training Loss: 4.678836822509766
Epoch: 0, Training Loss: 5.22426700592041
Epoch: 0, Training Loss: 3.174185276031494
Epoch: 0, Training Loss: 3.4368274211883545
Epoch: 0, Training Loss: 2.740471363067627
Epoch: 0, Training Loss: 3.059037446975708
Epoch: 0, Training Loss: 2.3849050998687744
Epoch: 0, Training Loss: 2.181574821472168
Epoch: 0, Training Loss: 2.767366647720337
Epoch: 0, Training Loss: 2.5764753818511963
Epoch: 0, Training Loss: 2.4283106327056885
Epoch: 0, Training Loss: 2.538130521774292
Epoch: 0, Training Loss: 2.8266284465789795
Epoch: 0, Training Loss: 2.524367332458496
Epoch: 0, Training Loss: 3.127093553543091
Epoch: 0, Training Loss: 2.474076509475708
Epoch: 0, Training Loss: 2.9501917362213135
Epoch: 0, Training Loss: 2.5646896362304688
Epoch: 0, Training Loss: 2.4305949211120605
Epoch: 0, Training Loss: 2.8118746280670166
Epoch: 0, Training Loss: 2.3225185871124268
Epoch: 0, Training Loss: 2.2575595378875732


## SAVE BEST MODEL

In [ ]:
save_directory = "/scratch/aa9798/best_model"
model.save_pretrained(save_directory)

# Optionally, save the tokenizer as well if needed
model_tokenizer = T5Tokenizer.from_pretrained("t5-base")
model_tokenizer.save_pretrained(save_directory)


## LOAD BEST MODEL

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Replace "/path/to/save/model" with the actual directory where you saved your model
saved_model_directory = "/scratch/aa9798/best_model"

# Load the model
model = T5ForConditionalGeneration.from_pretrained(saved_model_directory)

# If you also saved the tokenizer during training, load it as well
tokenizer = T5Tokenizer.from_pretrained(saved_model_directory)


## TRAIN VAL LOSS CURVE

In [ ]:
import matplotlib.pyplot as plt

# Example data (replace these lists with your actual data)
epochs = range(1, len(train_losses[:3]) + 1)


# Plotting the training loss
plt.plot(epochs, train_losses[:3], label='Training Loss', marker='o')
# Plotting the validation loss
plt.plot(epochs, val_losses[:3], label='Validation Loss', marker='o')

# Adding labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')

# Adding legend
plt.legend()

# Display the plot
plt.show()


## EVALUATION - COMPUTE METRICS

In [ ]:
rouge_scores = compute_rouge(actuals, predictions)
model = model.to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)



# Print Rouge scores
print('Epoch:', epoch + 1, '| Rouge-1:', round(rouge_scores['rouge-1']['f'], 4))
print('Epoch:', epoch + 1, '| Rouge-2:', round(rouge_scores['rouge-2']['f'], 4))
print('Epoch:', epoch + 1, '| Rouge-L:', round(rouge_scores['rouge-l']['f'], 4))
# Assuming you have a function to compute BLEU
bleu_score = compute_bleu(actuals, predictions)
print('Epoch:', epoch + 1, '| BLEU:', round(bleu_score, 4))
    

## PREDICTION 

In [ ]:
for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, test_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    print('Output Files generated for review')

In [ ]:
final_df.to_csv('/scratch/aa9798/final_df.csv')
